
---

# 📘 **FiftyOne 핵심 개념 & 메서드 요약**

---

## 🧩 **1. FiftyOne이란?**

FiftyOne은 **이미지·비디오 데이터셋을 시각화하고 분석하는 도구**로,
특히 **CV 모델 평가·오류 분석·EDA**에 매우 강력하다.

대표 기능:

* 이미지 그리드 시각화
* 라벨/예측 기반 필터링
* 혼동 행렬
* blur·brightness·해상도 기반 이상치 탐지
* 모델 예측 + GT 비교

---

# 🗂️ **Dataset 관련 메서드**

### ### 📌 `fo.Dataset(name)`

새로운 빈 Dataset 생성.

### 📌 `fo.Dataset.from_dir(...)`

폴더에서 이미지들을 자동으로 로드하여 Dataset 생성.
(필요하면 metadata 자동 생성되지만, 환경에 따라 안 될 때도 있음)

### 📌 `fo.load_dataset(name)`

이미 생성된 Dataset 불러오기.

### 📌 `fo.delete_dataset(name)`

동일한 Dataset 이름이 있을 때 삭제.

### 📌 `fo.list_datasets()`

현재 존재하는 모든 Dataset 이름 목록 확인.

---

# 🖼️ **Sample 관련 메서드**

### 📌 `fo.Sample(filepath=...)`

하나의 이미지 샘플 생성.

### 📌 `sample["predictions"] = fo.Classification(label="7")`

예측 결과를 추가하는 필드 생성.

* `"predictions"`는 필드명
* `Classification`은 라벨 형태의 데이터 타입

### 📌 `dataset.add_sample(sample)`

Dataset에 샘플 추가.

---

# 🧮 **Metadata 관련**

### 📌 `dataset.compute_metadata()`

이미지의 width, height, channels 등 **메타데이터 계산**.
→ blur/가로세로비/이상치 분석에 필수.

---

# 👀 **GUI 실행**

### 📌 `fo.launch_app(dataset)`

FiftyOne 웹 앱 실행.

### 📌 `session.open()`

브라우저에서 열기.

---

# 🔍 **Dataset Filtering 관련**

### 📌 `dataset.match(...)`

조건을 만족하는 subset(view)을 만드는 함수.
예시:

```python
view = dataset.match(fo.F("predictions.label") == "7")
```

→ prediction이 7인 이미지들만 반환.

---

# 📊 **평가(Evaluation)**

### 📌 `dataset.evaluate_classifications("predictions", gt_field="ground_truth")`

예측과 정답을 비교하여 정확도·혼동 행렬 계산.

> test에는 ground_truth가 없으므로 사용 불가
> valid에서만 사용

---

# 📁 **전체 요약**

| 기능            | 메서드                  | 설명              |
| ------------- | -------------------- | --------------- |
| Dataset 생성    | `fo.Dataset()`       | 빈 Dataset 만들기   |
| 폴더 로드         | `Dataset.from_dir()` | 폴더 → Dataset    |
| Sample 생성     | `fo.Sample()`        | 이미지 샘플 생성       |
| Prediction 추가 | `fo.Classification`  | 예측 라벨 필드        |
| Metadata 계산   | `compute_metadata()` | width/height 추출 |
| Dataset 필터링   | `match()`            | 조건 기반 subset    |
| GUI 실행        | `launch_app()`       | 시각화 앱 실행        |

---



In [1]:
import fiftyone as fo
import os


c:\Users\USER\.conda\envs\pytorch_test\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
# 이미 있는 Dataset을 재사용하기
dataset = fo.load_dataset("document-test")

In [ ]:
# 기존 Dataset을 지우고 새로 만들기
name = "document-test"

if name in fo.list_datasets():
    fo.delete_dataset(name)

dataset = fo.Dataset(name)

In [ ]:
train_dir = "./datasets/train"
test_dir = "./datasets/test"

# Dataset 생성

train_ds = fo.Dataset.from_dir(
    dataset_dir= train_dir,
    dataset_type=fo.types.ImageDirectory,
    name="document-train",
)

test_ds = fo.Dataset.from_dir(
    dataset_type=fo.types.ImageDirectory,
    dataset_dir = test_dir,
    name="document-test"
)



 100% |███████████████| 1570/1570 [257.1ms elapsed, 0s remaining, 6.2K samples/s]   
 100% |███████████████| 3140/3140 [305.8ms elapsed, 0s remaining, 10.3K samples/s]     


In [ ]:
# FiftyOne 실행
# compute_metadata() : meta 데이터를 함께 표시
train_ds.compute_metadata()
session = fo.launch_app(train_ds)

# 브라우저에서 열림
# session.open()     # 브라우저에서 열림


Could not connect session, trying again in 10 seconds



In [ ]:
# 한번의 작동 예시
import fiftyone as fo
import os

name = "document-test"

# 1) 있으면 삭제하고
if name in fo.list_datasets():
    fo.delete_dataset(name)

# 2) 새로 만들고
dataset = fo.Dataset(name)

test_dir = "./datasets/test"

for fname in os.listdir(test_dir):
    if fname.lower().endswith((".png", ".jpg", ".jpeg")):
        path = os.path.join(test_dir, fname)
        sample = fo.Sample(filepath=path)
        dataset.add_sample(sample)

# 3) 메타데이터 계산
dataset.compute_metadata()

# 4) 앱 실행
session = fo.launch_app(dataset)



Computing metadata...
 100% |███████████████| 3140/3140 [540.7ms elapsed, 0s remaining, 5.8K samples/s]      



Could not connect session, trying again in 10 seconds




Could not connect session, trying again in 10 seconds



In [ ]:
# 디버깅시 사용 예
import fiftyone as fo
import pandas as pd
import os

# CSV 경로
csv_path = "./output.csv"

# 테스트 이미지 폴더 경로
test_dir = "./datasets/test"

# CSV 읽기
df = pd.read_csv(csv_path)

# 데이터셋 이름
name = "document-test-pred"

# 기존 동일 이름 dataset 있으면 삭제
if name in fo.list_datasets():
    fo.delete_dataset(name)

# 새 데이터셋 생성
dataset = fo.Dataset(name)

# 샘플 추가
for _, row in df.iterrows():
    img_path = os.path.join(test_dir, row["ID"])

    sample = fo.Sample(filepath=img_path)
    sample["predictions"] = fo.Classification(label=str(row["target"]))
    dataset.add_sample(sample)

# 메타데이터 계산 (width/height)
dataset.compute_metadata()

# GUI 실행
session = fo.launch_app(dataset)



Computing metadata...
 100% |███████████████| 3140/3140 [525.4ms elapsed, 0s remaining, 6.0K samples/s]      



Could not connect session, trying again in 10 seconds

